## Bonus Points

##### a) Sort the users in terms of number of followers and divide them into two groups: for the first group, take only the top 10% regarding "followers", and for the second one, take the rest. Now compare the mean of time intervals between posts for the two categories. Do you notice something relevant?

In [3]:
#Data upload (posts.csv: 42,710.197 rows)
columns_posts = ["profile_id","cts"]
data_posts= pd.read_csv("~/instagram_posts.csv", sep = "\t" , usecols = columns_posts , parse_dates=["cts"])

In [2]:
#Data upload (profiles.csv: 4,509.586 rows)
columns_profiles = ["profile_id","followers"]
data_profiles = pd.read_csv("instagram_profiles.csv", sep = "\t" , usecols = columns_profiles)

1. In profiles.csv, we remove those rows that have missing values in "followers" and then we sort the data in ascending order by followers. 

In [19]:
data_profiles = data_profiles.dropna()
sorted_profiles = data_profiles.sort_values(by = "followers",ascending = False)

,profile_id,followers
3045705,1.524120e+07,2505.0
1793602,1.485528e+09,2505.0
2635421,2.087735e+08,2505.0
345129,1.237176e+09,2504.0
334101,6.864995e+09,2504.0
...,...,...
2125632,5.001772e+09,0.0
3435370,5.477267e+09,0.0
1323180,1.010826e+09,0.0
1902380,4.205987e+09,0.0


2. We create 2 new dataframes: one containing the top 10 percent of the sorted dataframe and the second with the remainig 90%.

In [ ]:
sorted_profiles_1 = sorted_profiles[0:int(len(sorted_profiles)*0.1+1)]
sorted_profiles_2 = sorted_profiles[int(len(sorted_profiles)*0.1+1):]

3. We <b>merge independently both datasets with posts.csv by profile_id</b>. As an output we have 2 new dataframes:
the first one contains all posts of the top_10 and the other contains all posts of the bottom 90 users (based on followers).

In [37]:
top_10_followers = pd.merge(sorted_profiles_1, data_posts, on='profile_id')
bottom_90_follow = pd.merge(sorted_profiles_2, data_posts, on='profile_id')
top_10_followers

,profile_id,followers,cts
0,2.502532e+07,285457645.0,2012-12-01 17:42:03
1,2.502532e+07,285457645.0,2018-01-17 20:59:30
2,2.502532e+07,285457645.0,2018-12-20 03:03:13
3,2.502532e+07,285457645.0,2017-09-19 03:56:12
4,2.502532e+07,285457645.0,2014-10-29 14:21:30
...,...,...,...
3981521,7.186242e+09,2505.0,2019-08-13 08:39:16
3981522,7.186242e+09,2505.0,2019-08-13 08:36:35
3981523,7.186242e+09,2505.0,2019-08-13 08:34:27
3981524,7.186242e+09,2505.0,2019-08-09 12:01:49


4. We <b>group by profile_id</b> both datasets and <b>aggregate a new column containing the average time bettwen post per each user. We also remove those users who only made one post, as they are not relevant for our analysis. Below we can see both outputs:

In [67]:
top_10_grouped = top_10_followers.groupby("profile_id").aggregate({'followers': "max",'cts':["max", "min", "count"]})
top_10_grouped =  top_10_grouped[top_10_grouped.cts["count"] > 1]
top_10_grouped["average_time_between_posts"] = (top_10_grouped.cts['max'] - top_10_grouped.cts['min']) / (top_10_grouped.cts['count']-1)

bot_90_grouped = bottom_90_follow.groupby("profile_id").aggregate({'followers': "max",'cts':["max", "min", "count"]},"followers")
bot_90_grouped =  bot_90_grouped[bot_90_grouped.cts["count"] > 1]
bot_90_grouped["average_time_between_posts"] = (bot_90_grouped.cts['max'] - bot_90_grouped.cts['min']) / (bot_90_grouped.cts['count']-1)

#TOP 10% OF USERS DATAFRAME WITH AVERAGE TIME BETWEEN POSTS PER USER
top_10_grouped

followers                 cts                            \
                    max                 max                 min count   
profile_id                                                              
4.000000e+00  4937112.0 2019-05-05 08:03:36 2010-07-22 12:19:51    37   
1.900000e+01   119775.0 2019-04-17 07:44:13 2010-10-23 18:48:33    51   
2.800000e+01   113310.0 2019-04-16 05:10:12 2011-02-19 20:16:29    92   
3.400000e+01    83915.0 2019-07-01 20:49:50 2011-07-29 06:24:16    20   
5.600000e+01    40587.0 2016-07-25 05:36:50 2015-01-18 03:38:06     3   
...                 ...                 ...                 ...   ...   
1.349627e+10     5123.0 2019-07-05 16:11:17 2019-05-10 06:29:52    13   
1.350481e+10     2646.0 2019-07-07 19:58:19 2019-05-08 14:33:27    18   
1.351643e+10     4771.0 2019-08-12 13:17:36 2019-05-10 16:15:26    14   
1.370178e+10     2555.0 2019-07-17 18:34:39 2019-05-17 10:15:10    13   
1.418136e+10     9175.0 2019-06-20 16:06:28 2019-06-06 23:46:14    15   

              average_time_between_posts  
                                          
profile_id                                
4.000000e+00  89 days 03:12:52.916666666  
1.900000e+01     61 days 22:49:06.800000  
2.800000e+01  32 days 17:14:26.186813186  
3.400000e+01 152 days 08:20:17.578947368  
5.600000e+01           277 days 00:59:22  
...                                  ...  
1.349627e+10   4 days 16:48:27.083333333  
1.350481e+10   3 days 13:01:27.764705882  
1.351643e+10   7 days 05:18:37.692307692  
1.370178e+10   5 days 02:41:37.416666666  
1.418136e+10   0 days 23:27:09.571428571  

[269112 rows x 5 columns]

In [68]:
#BOTTOM 90% OF USERS DATAFRAME WITH AVERAGE TIME BETWEEN POSTS PER USER
bot_90_grouped

followers                 cts                            \
                   max                 max                 min count   
profile_id                                                             
1.640000e+02    1582.0 2019-06-22 20:05:06 2019-02-11 11:47:46    13   
2.080000e+02     381.0 2019-03-27 04:55:36 2019-03-14 04:37:46    12   
2.660000e+02     845.0 2019-02-05 16:06:02 2018-09-20 03:51:13    12   
4.920000e+02     671.0 2019-05-19 15:13:40 2019-03-10 12:37:20    14   
5.600000e+02     132.0 2019-08-05 22:52:00 2019-04-29 08:52:41    13   
...                ...                 ...                 ...   ...   
1.499738e+10     786.0 2019-07-11 04:25:20 2019-07-09 03:18:33    12   
1.501918e+10     516.0 2019-07-21 14:05:34 2019-06-19 13:17:00     5   
1.523330e+10       4.0 2019-06-27 00:11:09 2019-06-26 23:56:47     3   
1.566695e+10      49.0 2019-08-06 01:01:31 2019-07-29 21:36:32    11   
9.001013e+13      46.0 2018-04-04 09:53:28 2017-06-22 10:56:54    10   

             average_time_between_posts  
                                         
profile_id                               
1.640000e+02 10 days 22:41:26.666666666  
2.080000e+02  1 days 04:23:26.363636363  
2.660000e+02 12 days 14:12:15.363636363  
4.920000e+02  5 days 09:25:52.307692307  
5.600000e+02  8 days 05:09:56.583333333  
...                                 ...  
1.499738e+10  0 days 04:27:53.363636363  
1.501918e+10     8 days 00:12:08.500000  
1.523330e+10            0 days 00:07:11  
1.566695e+10     0 days 17:08:29.900000  
9.001013e+13 31 days 18:32:57.111111111  

[1690690 rows x 5 columns]

5. We <b>compute the mean of the "average time between posts" per user, for each one of the datasets:

In [69]:
print("Top 10% of users in terms of followers post on average every",top_10_grouped["average_time_between_posts"].mean())
print("Bottom 90% of users in terms of followers post on average every ",bot_90_grouped["average_time_between_posts"].mean())

Top 10% of users in terms of followers post on average every 28 days 17:43:41.810042290
Bottom 90% of users in terms of followers post on average every  23 days 17:58:06.043257518


Surprisingly, <b>users with more followers post on average less often</b> (around every 28 days) <b>than the users with less followers</b> (around every 23 days). As this result was strange to us (one would expect the opposite, thinking of "influencers" for instance), <b>we decided to further investigate the distributions of average times</b> to find if maybe there could be outliers that would overestimate the means of both distributions:

In [86]:
print("Summary Statistics: Top 10%")
print("")
print(top_10_grouped.average_time_between_posts.describe())
print("")
print("Percentile       Quantile")
print(top_10_grouped.average_time_between_posts.quantile([.5, .90, .95 , .99 , .999 , .9999 , .999999 , 1]))

Summary Statistics: Top 10%

count                        269112
mean     28 days 17:43:41.810042290
std      64 days 10:14:36.071104174
min          0 days 00:00:02.800000
25%       3 days 18:16:40.458333333
50%       9 days 23:27:35.572192513
75%      34 days 10:33:53.113636363
max              2400 days 03:24:34
Name: average_time_between_posts, dtype: object

Percentile       Quantile
0.500000      9 days 23:27:35.572192513
0.900000     71 days 14:39:54.128571429
0.950000     97 days 22:09:17.752424242
0.990000    210 days 16:56:36.748333388
0.999000    917 days 21:21:00.513500704
0.999900   1623 days 20:30:52.706580512
0.999999   2375 days 22:38:54.763439872
1.000000             2400 days 03:24:34
Name: average_time_between_posts, dtype: timedelta64[ns]


In [85]:
print("Summary Statistics: Bottom 90%")
print("")
print(bot_90_grouped.average_time_between_posts.describe())
print("")
print("Percentile       Quantile")
print(bot_90_grouped.average_time_between_posts.quantile([.5, .90, .95 , .99 , .999 , .9999 , .999999 , 1]))

Summary Statistics: Bottom 90%

count                       1690690
mean     23 days 17:58:06.043257518
std      47 days 21:17:47.545413374
min                 0 days 00:00:00
25%       4 days 17:02:26.464285713
50%      10 days 14:25:28.909090908
75%      26 days 13:20:57.545454545
max              2890 days 18:18:06
Name: average_time_between_posts, dtype: object

Percentile       Quantile
0.500000     10 days 14:25:28.909090908
0.900000     57 days 02:39:14.327272737
0.950000     81 days 12:06:46.249999992
0.990000    173 days 18:33:42.128285628
0.999000    612 days 16:58:03.135010288
0.999900   1406 days 14:17:49.103562336
0.999999   2541 days 22:37:41.516439776
1.000000             2890 days 18:18:06
Name: average_time_between_posts, dtype: timedelta64[ns]


 We can observe outliers i both distributions that if we remove, we will obtain a more realistic average time between posts for each group. For example if we exclude the top 1% of users in terms of average time between post for each group we obtain more representative mean values:

In [89]:
top_10_grouped_reduced =  top_10_grouped[top_10_grouped.average_time_between_posts <= "210 days 16:56:36.748333388"]
bot_90_grouped_reduced =  bot_90_grouped[bot_90_grouped.average_time_between_posts <= "173 days 18:33:42.128285628"]

print("Top 10% of users in terms of followers post on average every",bot_90_grouped["average_time_between_posts"].mean())
print("Bottom 90% of users in terms of followers post on average every",bot_90_grouped_reduced["average_time_between_posts"].mean())

Top 10% of users in terms of followers post on average every 23 days 17:58:06.043257518
Bottom 90% of users in terms of followers post on average every 20 days 12:14:17.387364731


After comparing both distributions witouth outliers, they actually show that <b>users with less followers on average post more often</b>. So we could have as a plausible hypothesis that, on average, people who have less followers might feel less pressured to post only "top content" and might actually post more frequently, not being so concious about what to post (i.e. having a small but close circle of friends)